In [203]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
df = pd.read_csv("Run_scorers.csv")

In [205]:
df

,id,name,span,Matches,Innings,Not_outs,Runs,High_score,Avg,Centuries,Half_centuries,Ducks
0,1,SR Tendulkar (IND),1989-2013,200,329,33,15921,248*,53.78,51,68,14
1,2,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17
2,3,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,55.37,45,58,16
3,4,R Dravid (ICC/IND),1996-2012,164,286,32,13288,270,52.31,36,63,8
4,5,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9
...,...,...,...,...,...,...,...,...,...,...,...,...
97,98,JH Edrich (ENG),1963-1976,77,127,9,5138,310*,43.54,12,24,6
98,99,A Ranatunga (SL),1982-2000,93,155,12,5105,135*,35.69,4,38,12
99,100,Tamim Iqbal (BAN),2008-2022,69,132,2,5082,206,39.09,10,31,11
100,101,KC Brathwaite (WI),2011-2022,80,154,10,5067,212,35.18,11,28,15


In [206]:
df.dropna(inplace=True)

In [207]:
df.describe()

,id,Matches,Innings,Not_outs,Runs,Avg,Centuries,Half_centuries,Ducks
count,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000
mean,51.500000,104.784314,179.107843,15.803922,7526.549020,46.230588,20.215686,35.362745,11.588235
std,29.588849,26.595476,44.055075,8.730326,2232.930196,8.218666,8.315115,11.380711,4.030461
min,1.000000,52.000000,80.000000,2.000000,5062.000000,30.300000,4.000000,13.000000,4.000000
25%,26.250000,86.250000,148.750000,10.000000,5765.000000,41.995000,14.000000,28.000000,9.000000
50%,51.500000,101.500000,177.000000,14.500000,7141.000000,45.205000,19.000000,33.000000,12.000000
75%,76.750000,117.000000,199.500000,20.000000,8442.500000,50.187500,24.000000,40.750000,14.000000
max,102.000000,200.000000,329.000000,49.000000,15921.000000,99.940000,51.000000,68.000000,22.000000


In [208]:
df['Runs'].max()

15921

In [209]:
df.loc[df['Runs'] == df['Runs'].max()]

,id,name,span,Matches,Innings,Not_outs,Runs,High_score,Avg,Centuries,Half_centuries,Ducks
0,1,SR Tendulkar (IND),1989-2013,200,329,33,15921,248*,53.78,51,68,14


In [210]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              102 non-null    int64  
 1   name            102 non-null    object 
 2   span            102 non-null    object 
 3   Matches         102 non-null    int64  
 4   Innings         102 non-null    int64  
 5   Not_outs        102 non-null    int64  
 6   Runs            102 non-null    int64  
 7   High_score      102 non-null    object 
 8   Avg             102 non-null    float64
 9   Centuries       102 non-null    int64  
 10  Half_centuries  102 non-null    int64  
 11  Ducks           102 non-null    int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 9.7+ KB


In [211]:
df.corr()

,id,Matches,Innings,Not_outs,Runs,Avg,Centuries,Half_centuries,Ducks
id,1.000000,-0.753209,-0.795751,-0.490756,-0.909782,-0.407565,-0.826455,-0.753566,-0.193525
Matches,-0.753209,1.000000,0.969833,0.675995,0.850079,-0.013202,0.635253,0.840425,0.414906
Innings,-0.795751,0.969833,1.000000,0.629256,0.888317,-0.024192,0.664717,0.872388,0.421469
Not_outs,-0.490756,0.675995,0.629256,1.000000,0.567459,0.214548,0.451082,0.648052,0.199995
Runs,-0.909782,0.850079,0.888317,0.567459,1.000000,0.390064,0.901943,0.836637,0.215211
Avg,-0.407565,-0.013202,-0.024192,0.214548,0.390064,1.000000,0.587134,0.123317,-0.313303
Centuries,-0.826455,0.635253,0.664717,0.451082,0.901943,0.587134,1.000000,0.582666,0.111690
Half_centuries,-0.753566,0.840425,0.872388,0.648052,0.836637,0.123317,0.582666,1.000000,0.219140
Ducks,-0.193525,0.414906,0.421469,0.199995,0.215211,-0.313303,0.111690,0.219140,1.000000


In [184]:
plt.clf()
plt.figure(figsize=(12, 9))
ax = plt.subplot(111)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()
plt.xticks(fontsize=14)
plt.yticks(range(5000, 30001, 5000), fontsize=14)
plt.xlabel("Total Career Runs", fontsize=16)
plt.ylabel("Count", fontsize=16)
plt.hist(df['Runs'], color="#3F5D7D", bins=100)

(array([5., 5., 4., 6., 3., 0., 5., 2., 2., 1., 5., 1., 1., 0., 0., 2., 2.,
        4., 3., 3., 2., 2., 5., 1., 4., 1., 0., 3., 2., 1., 1., 1., 4., 0.,
        4., 1., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 1.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 2., 1., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 0., 2., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([ 5062.  ,  5170.59,  5279.18,  5387.77,  5496.36,  5604.95,
         5713.54,  5822.13,  5930.72,  6039.31,  6147.9 ,  6256.49,
         6365.08,  6473.67,  6582.26,  6690.85,  6799.44,  6908.03,
         7016.62,  7125.21,  7233.8 ,  7342.39,  7450.98,  7559.57,
         7668.16,  7776.75,  7885.34,  7993.93,  8102.52,  8211.11,
         8319.7 ,  8428.29,  8536.88,  8645.47,  8754.06,  8862.65,
         8971.24,  9079.83,  9188.42,  9297.01,  9405.6 ,  9514.19,
         9622.78,  9731.37,  9839.96,  9948.55, 10057.14, 10165.73,
      

In [212]:
df.count()

id                102
name              102
span              102
Matches           102
Innings           102
Not_outs          102
Runs              102
High_score        102
Avg               102
Centuries         102
Half_centuries    102
Ducks             102
dtype: int64

In [186]:
plt.clf()
plt.scatter(df["Half_centuries"], df["Centuries"])
plt.xlabel("Half Centuries", fontsize=16)
plt.ylabel("Centuries", fontsize=16)
plt.show()

In [213]:
df

,id,name,span,Matches,Innings,Not_outs,Runs,High_score,Avg,Centuries,Half_centuries,Ducks
0,1,SR Tendulkar (IND),1989-2013,200,329,33,15921,248*,53.78,51,68,14
1,2,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17
2,3,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,55.37,45,58,16
3,4,R Dravid (ICC/IND),1996-2012,164,286,32,13288,270,52.31,36,63,8
4,5,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9
...,...,...,...,...,...,...,...,...,...,...,...,...
97,98,JH Edrich (ENG),1963-1976,77,127,9,5138,310*,43.54,12,24,6
98,99,A Ranatunga (SL),1982-2000,93,155,12,5105,135*,35.69,4,38,12
99,100,Tamim Iqbal (BAN),2008-2022,69,132,2,5082,206,39.09,10,31,11
100,101,KC Brathwaite (WI),2011-2022,80,154,10,5067,212,35.18,11,28,15


In [214]:
df["Half_century_percentage"] = df["Half_centuries"] / df["Innings"]
df["Century_percentage"] = df["Centuries"] / df["Innings"]
df["Duck_percentage"] = df["Ducks"] / df["Innings"]

In [215]:
df

,id,name,span,Matches,Innings,Not_outs,Runs,High_score,Avg,Centuries,Half_centuries,Ducks,Half_century_percentage,Century_percentage,Duck_percentage
0,1,SR Tendulkar (IND),1989-2013,200,329,33,15921,248*,53.78,51,68,14,0.206687,0.155015,0.042553
1,2,RT Ponting (AUS),1995-2012,168,287,29,13378,257,51.85,41,62,17,0.216028,0.142857,0.059233
2,3,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,55.37,45,58,16,0.207143,0.160714,0.057143
3,4,R Dravid (ICC/IND),1996-2012,164,286,32,13288,270,52.31,36,63,8,0.220280,0.125874,0.027972
4,5,AN Cook (ENG),2006-2018,161,291,16,12472,294,45.35,33,57,9,0.195876,0.113402,0.030928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,98,JH Edrich (ENG),1963-1976,77,127,9,5138,310*,43.54,12,24,6,0.188976,0.094488,0.047244
98,99,A Ranatunga (SL),1982-2000,93,155,12,5105,135*,35.69,4,38,12,0.245161,0.025806,0.077419
99,100,Tamim Iqbal (BAN),2008-2022,69,132,2,5082,206,39.09,10,31,11,0.234848,0.075758,0.083333
100,101,KC Brathwaite (WI),2011-2022,80,154,10,5067,212,35.18,11,28,15,0.181818,0.071429,0.097403


In [227]:
df_binary = df[['Century_percentage','Avg']]
df_other = df[['Half_century_percentage','Avg']]

In [229]:
plt.clf()
plt.scatter(df_binary["Century_percentage"], df_binary["Avg"])
plt.xlabel("Century Percentage", fontsize=16)
plt.ylabel("Average", fontsize=16)
plt.show()

In [230]:
plt.clf()
plt.scatter(df_other["Half_century_percentage"], df_other["Avg"])
plt.xlabel("Half Century Percentage", fontsize=16)
plt.ylabel("Average", fontsize=16)
plt.show()

In [243]:
X = np.array(df_binary['Century_percentage']).reshape(-1,1)
Y = np.array(df_binary['Avg']).reshape(-1,1)
print("X: ", X)
print("Y: ", Y)

X:  [[0.1550152 ]
 [0.14285714]
 [0.16071429]
 [0.12587413]
 [0.11340206]
 [0.16309013]
 [0.14655172]
 [0.10714286]
 [0.13492063]
 [0.10188679]
 [0.12307692]
 [0.12173913]
 [0.1588785 ]
 [0.15962441]
 [0.13023256]
 [0.13170732]
 [0.09302326]
 [0.12169312]
 [0.125     ]
 [0.07555556]
 [0.11518325]
 [0.14141414]
 [0.16304348]
 [0.12777778]
 [0.13186813]
 [0.06382979]
 [0.18589744]
 [0.08823529]
 [0.12707182]
 [0.11398964]
 [0.15606936]
 [0.1625    ]
 [0.09569378]
 [0.13483146]
 [0.0754717 ]
 [0.10731707]
 [0.12637363]
 [0.09693878]
 [0.11702128]
 [0.1027027 ]
 [0.15384615]
 [0.10215054]
 [0.10857143]
 [0.08910891]
 [0.11052632]
 [0.15584416]
 [0.11931818]
 [0.15714286]
 [0.08241758]
 [0.08510638]
 [0.04761905]
 [0.1589404 ]
 [0.10674157]
 [0.11797753]
 [0.3625    ]
 [0.07446809]
 [0.13768116]
 [0.07303371]
 [0.09189189]
 [0.15267176]
 [0.1097561 ]
 [0.08938547]
 [0.06818182]
 [0.12578616]
 [0.13868613]
 [0.10948905]
 [0.14965986]
 [0.09090909]
 [0.15328467]
 [0.09032258]
 [0.08860759]
 [

In [256]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)
regr = LinearRegression()
regr.fit(X_train, Y_train)
regr.score(X_test, Y_test)

0.8774604626298671

In [258]:
plt.clf()
y_pred = regr.predict(X_test)
plt.xlabel("Century Percentage", fontsize=16)
plt.ylabel("Average", fontsize=16)
plt.scatter(X_test, Y_test, color='b')
plt.plot(X_test, y_pred, color='g')
plt.show()